In [ ]:
!pip install dask[complete] pandas pyarrow --quiet

import dask.bag as db
import dask.dataframe as dd
import pandas as pd
import json
import math

DATASET_PATH = "hf://datasets/Muennighoff/natural-instructions/**/train/*.jsonl"
TASK_MAP_FILE = "task_to_category_map.json"  
OUTPUT_PATH = "natural_instructions_sample_balanced.parquet"
TARGET_SIZE = 100_000
SEED = 42
REPARTITIONS = 200

with open(TASK_MAP_FILE, "r") as f:
    task_name_to_family = json.load(f)

print("Reading dataset lazily...")
bag = db.read_text(DATASET_PATH)

def safe_parse(line):
    try:
        ex = json.loads(line)
        task_name = ex.get("task_name", "")
        return {
            "task_name": task_name,
            "task_family": task_name_to_family.get(task_name, "other"),
            "id": ex.get("id", ""),
            "definition": str(ex.get("definition", "")),
            "inputs": str(ex.get("inputs", "")),
            "targets": str(ex.get("targets", "")),
        }
    except Exception:
        return None

parsed = bag.map(safe_parse).filter(lambda x: x is not None)

df = parsed.to_dataframe()

df = df.repartition(npartitions=REPARTITIONS)

cols = ["task_name", "task_family", "id", "definition", "inputs", "targets"]
df_pd = df[cols].compute()
print(f"Total records loaded: {len(df_pd):,}")

task_names = df_pd["task_name"].dropna().unique()
rows_per_task = max(1, TARGET_SIZE // len(task_names))
print(f"Sampling ~{rows_per_task} rows per task from {len(task_names)} unique task_names")

sampled_list = []

for task in task_names:
    task_df = df_pd[df_pd["task_name"] == task]
    n = min(len(task_df), rows_per_task)  # avoid sampling more than available
    if n == 0:
        continue  # task_name exists but has no rows (just in case)
    sampled_list.append(task_df.sample(n=n, random_state=SEED))

# Concatenate all samples
sampled_df = pd.concat(sampled_list, ignore_index=True)

# Shuffle final sample
sampled_df = sampled_df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
print(f"Final sampled size: {len(sampled_df):,} rows")

sampled_df.to_parquet(OUTPUT_PATH, index=False)
print(f"Saved sampled dataset to {OUTPUT_PATH}")

# Preview
print(sampled_df[["task_name", "task_family"]].head(10))


Reading dataset lazily...
Total records loaded: 3,082,094
Sampling ~132 rows per task from 757 unique task_names
Final sampled size: 98,417 rows
Saved sampled dataset to natural_instructions_sample_balanced.parquet
                                           task_name          task_family
0            task073_commonsenseqa_answer_generation   Question Answering
1                    task846_pubmedqa_classification  Answer Verification
2              task096_conala_list_index_subtraction    Program Execution
3    task165_mcscript_question_answering_commonsense   Question Answering
4                      task177_para-nmt_paraphrasing         Paraphrasing
5                    task560_alt_translation_en_entk          Translation
6             task1603_smcalflow_sentence_generation  Dialogue Generation
7  task047_miscellaneous_answering_science_questions   Question Answering
8                  task211_logic2text_classification         Text to Code
9          task862_asdiv_multidiv_question_an